In [1]:
import baltic as bt
import pandas as pd
import json 

tree_path = "/Users/monclalab1/Documents/moncla/non-human-h3/auspice/nonhuman_influenza_h3nx_ha.json"

mytree, mymeta = bt.loadJSON(tree_path)



Tree height: 81.552803
Tree length: 8664.777550
annotations present

Numbers of objects in tree: 6453 (2940 nodes and 3513 leaves)



In [3]:
#this parses the tree and returns a list of strain names that are in the swine, equine, and canine lineages
#with the intent of making a new nextstrain build that is host specific

count=0
node_list = []
leaf_list =[]

for k in mytree.Objects[1:]:
        if k.branchType == "node":
            if k.traits["node_attrs"]["host"]["value"] != k.parent.traits["node_attrs"]["host"]["value"]:
                if len(k.leaves) >= 10:
                    new = mytree.subtree(k = k)
                    for i in new.Objects:
                        if i.branchType == "leaf":
                            if i.traits["node_attrs"]["host"]["value"] == new.root.traits["node_attrs"]["host"]["value"]:
                                count += 1
                                if count >=10:
                                    node_list.append(k) #store root node because that node is now the start of a new lineage
                                    count=0
                                    break

                
equine = mytree.subtree(k=(node_list[0]))
equine_list = []
for i in equine.Objects:
        if i.branchType == "leaf":
            leaf_list.append(i.traits["name"])
            if i.traits["node_attrs"]["host"]["value"] == equine.root.traits["node_attrs"]["host"]["value"]:    
                equine_list.append(i.traits["name"])
                
EquCanine = mytree.subtree(k=(node_list[1]))
EquCanine_list = []
for j in EquCanine.Objects:
        if j.branchType == "leaf":
            leaf_list.append(j.traits["name"])
            if j.traits["node_attrs"]["host"]["value"] == EquCanine.root.traits["node_attrs"]["host"]["value"]:    
                EquCanine_list.append(j.traits["name"])

canine = mytree.subtree(k=(node_list[2]))
canine_list = []
for h in canine.Objects:
        if h.branchType == "leaf":
            leaf_list.append(h.traits["name"])
            if h.traits["node_attrs"]["host"]["value"] == canine.root.traits["node_attrs"]["host"]["value"]:    
                canine_list.append(h.traits["name"])

swine = mytree.subtree(k=(node_list[3]))
swine_list = []

for g in swine.Objects:
        if g.branchType == "leaf":
            leaf_list.append(g.traits["name"])
            if g.traits["node_attrs"]["host"]["value"] == swine.root.traits["node_attrs"]["host"]["value"]:    
                swine_list.append(g.traits["name"])
                


#combining equine and equcanine:
Equine_Canine_Clade = EquCanine_list + equine_list
print(len(Equine_Canine_Clade))
avian_list =[]

for k in mytree.Objects:
        if k.branchType == "leaf":
            if k.traits["name"] not in leaf_list and k.traits["node_attrs"]["host"]["value"] == "Avian":
                avian_list.append(k.traits["name"])
          


431


In [5]:
#subsetting the existing fasta file to only have the strains from the specific host
fasta_file = "/Users/monclalab1/Documents/moncla/non-human-h3/h3nx_seg_filtered_and_integrated/h3nx_pb1.fa"

fasta_data = []

with open(fasta_file) as f:
    header = ""
    sequence = ""
    for line in f:
        if line.startswith(">"):
            if header != "":
                fasta_data.append({"header": header, "sequence": sequence})
            header = line.strip()
            sequence = ""
        else:
            sequence += line.strip()
    fasta_data.append({"header": header, "sequence": sequence})

df = pd.DataFrame(fasta_data)

df['Strain'] = df['header'].str.split("|").str[0]
df['Strain'] = df['Strain'].str[1:]


#change this to get the correct host
df = df[df["Strain"].isin(equine_list)]

df = df.drop(["Strain"], axis = 1)

#change this to name the file correctly
host_fasta_file = "/Users/monclalab1/Documents/moncla/non-human-h3/host-specific/equine_pb1.fasta"
with open(host_fasta_file, "w") as f:
    for index, row in df.iterrows():
        f.write(f"{row['header']}\n")
        f.write(f"{row['sequence']}\n")

